In [1]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

import scipy

import qgrid
import pandas as pd

In [3]:
#data taken from https://en.wikipedia.org/wiki/Van_der_Waals_constants_(data_page) using https://wikitable2csv.ggor.de/
#format: Element, a (L2bar/mol2), b (L/mol)

raw_data = [
"Acetic acid",17.71,0.1065,
"Acetic anhydride",20.158,0.1263,
"Acetone",16.02,0.1124,
"Acetonitrile",17.81,0.1168,
"Acetylene",4.516,0.0522,
"Ammonia",4.225,0.0371,
"Argon",1.355,0.03201,
"Benzene",18.24,0.1154,
"Bromobenzene",28.94,0.1539,
"Butane",14.66,0.1226,
"Carbon dioxide",3.640,0.04267,
"Carbon disulfide",11.77,0.07685,
"Carbon monoxide",1.505,0.03985,
"Carbon tetrachloride",19.7483,0.1281,
"Chlorine",6.579,0.05622,
"Chlorobenzene",25.77,0.1453,
"Chloroethane",11.05,0.08651,
"Chloromethane",7.570,0.06483,
"Cyanogen",7.769,0.06901,
"Cyclohexane",23.11,0.1424,
"Diethyl ether",17.61,0.1344,
"Diethyl sulfide",19.00,0.1214,
"Dimethyl ether",8.180,0.07246,
"Dimethyl sulfide",13.04,0.09213,
"Ethane",5.562,0.0638,
"Ethanethiol",11.39,0.08098,
"Ethanol",12.18,0.08407,
"Ethyl acetate",20.72,0.1412,
"Ethylamine",10.74,0.08409,
"Fluorobenzene",20.19,0.1286,
"Fluoromethane",4.692,0.05264,
"Freon",10.78,0.0998,
"Germanium tetrachloride",22.90,0.1485,
"Helium",0.0346,0.0238,
"Hexane",24.71,0.1735,
"Hydrogen",0.2476,0.02661,
"Hydrogen bromide",4.510,0.04431,
"Hydrogen chloride",3.716,0.04081,
"Hydrogen selenide",5.338,0.04637,
"Hydrogen sulfide",4.490,0.04287,
"Iodobenzene",33.52,0.1656,
"Krypton",2.349,0.03978,
"Mercury",8.200,0.01696,
"Methane",2.283,0.04278,
"Methanol",9.649,0.06702,
"Neon",0.2135,0.01709,
"Nitric oxide",1.358,0.02789,
"Nitrogen",1.370,0.0387,
"Nitrogen dioxide",5.354,0.04424,
"Nitrous oxide",3.832,0.04415,
"Oxygen",1.382,0.03186,
"Pentane",19.26,0.146,
"Phosphine",4.692,0.05156,
"Propane",8.779,0.08445,
"Radon",6.601,0.06239,
"Silane",4.377,0.05786,
"Silicon tetrafluoride",4.251,0.05571,
"Sulfur dioxide",6.803,0.05636,
"Tin tetrachloride",27.27,0.1642,
"Toluene",24.38,0.1463,
"Water",5.536,0.03049,
"Xenon",4.250,0.05105
]

In [4]:
data_array = np.array(raw_data)
data_reshaped = np.reshape(data_array, (-1,3)); #reshape in three columns

In [5]:
database = pd.DataFrame(data=data_reshaped, columns=["Element", "a (L2bar/mol2)", "b (L/mol)"])

#numpy converts all elements to 'object' and pandas interprets them as string, but I want a and b values to be float
database["a (L2bar/mol2)"] = np.round(pd.to_numeric(database["a (L2bar/mol2)"]), 4)
database["b (L/mol)"] = np.round(pd.to_numeric(database["b (L/mol)"]), 5)

In [6]:
def calculate_critic(a, b):
    
    """
        This function calculates the critic point 
        (p_c, v_c, T_c) from given a and b parameters of 
        the Van der Waals equation of state for real gases.
        
        :math:`(P + a \\frac{n^2}{V^2})(V - nb) = nRT`
        
        :math:`p_c = \\frac{a}{27 b^2}`
        :math:`v_c = 3b`
        :math:`T_c = \\frac{8a}{27 b R}`
        
   Args:
       a: Term related with the attraction between particles in
       L^2 bar/mol^2.\n
       b: Term related with the volume that is occupied by one 
       mole of the molecules in L/mol.\n
       
   Returns:
       p_c: Critical pressure in bar.\n
       v_c: Critical volume in L/mol.\n
       T_c: Critical tenperature in K.\n
        
    """
    
    if b == 0.0:
        return None
    
    k_B = 1.3806488e-23 #m^2 kg s^-2 K^-1
    N_A = 6.02214129e23 
    R = 0.082 * 1.01325 #atm L mol^-1 K^-1 #now in bar L mol^-1 K^-1
    
    p_c = a/27.0/(b**2)
    v_c = 3.0*b
    T_c = 8.0*a/27.0/b/R
    
    return p_c, v_c, T_c

In [7]:
def generate_critic_points(df):
    #takes a dataframe containing three columns (element, a, b) and returns four arrays: pc, vc, Tc and names
    
    pc = []
    vc = []
    Tc = []
    
    names = []
    
    name_values = df.iloc[:,0]
    a_values, b_values, names_sorted = get_a_b_names(df)
    
    #update_slider_options(a_values, b_values)
    
    #name_values_sorted = [x for x,_ in sorted(zip(name_values, a_values), key=lambda x: float(x[1]), reverse=False)]
    #print(name_values_sorted)
    
    for i in range(len(a_values)):

        names.append(names_sorted[i])
        
        a = float(a_values[i])
        b = float(b_values[i])       
        p, v, T = calculate_critic(a, b)
        
        pc.append(p)
        vc.append(v)
        Tc.append(T)

    return pc, vc, Tc, names

In [8]:
def get_absolute_isotherms(a, b, v_values, T_values):
    #given v (1d array) and T (Nd array) calculate N isotherm arrays 
    
    isotherms = []
    
    #N_A = 6.02214129e23 

    R = 0.082 * 1.01325 #atm L mol^-1 K^-1 #now in bar L mol^-1 K^-1

    for T in T_values:
        isot = []
        for v in v_values:
            p = R*T/(v - b) - (a/v**2)
            isot.append(p)
            
        isotherms.append(isot)
        
    return isotherms

In [9]:
def bar_to_atm(p_values):
    
    for i in range(len(p_values)):
        p_values[i] = np.array(p_values[i]) * 0.9869 #unit change from bar to atm
        
    return p_values

In [10]:
def change_visible_points(a, b):
    p_c, v_c, T_c, names = generate_critic_points(qgrid_table.get_changed_df())
    
    fig_113_002.axes[0].scale = bqs.LinearScale(min = min(v_c), max = max(v_c))
    fig_113_002.axes[1].scale = bqs.LinearScale(min = min(p_c), max = max(p_c))
    
    for mark in fig_113_002.marks:
        mark.scales = {'x': bqs.LinearScale(min = min(v_c), max = max(v_c)), 'y': bqs.LinearScale(min = min(p_c), max = max(p_c))}

    fig_113_003.axes[0].scale = bqs.LinearScale(min = min(v_c), max = max(v_c))
    fig_113_003.axes[1].scale = bqs.LinearScale(min = min(T_c), max = max(T_c))
    
    for mark in fig_113_003.marks:
        mark.scales = {'x': bqs.LinearScale(min = min(v_c), max = max(v_c)), 'y': bqs.LinearScale(min = min(T_c), max = max(T_c))}
    
    fig_113_002.marks[0].x, fig_113_002.marks[0].y, fig_113_002.marks[0].names = v_c, p_c, names
    fig_113_003.marks[0].x, fig_113_003.marks[0].y, fig_113_003.marks[0].names = v_c, T_c, names
    
    a_values, b_values, names_sorted = get_a_b_names(qgrid_table.get_changed_df())
    
    fig_113_004.axes[0].scale = bqs.LinearScale(min = min(b_values), max = max(b_values))
    fig_113_004.axes[1].scale = bqs.LinearScale(min = min(a_values), max = max(a_values))
    
    for mark in fig_113_004.marks:
        mark.scales = {'x': bqs.LinearScale(min = min(b_values), max = max(b_values)),
                       'y': bqs.LinearScale(min = min(a_values), max = max(a_values))}
    
    update_slider_options(a_values, b_values)
    #a_values, b_values = get_a_b_values(qgrid_table.get_changed_df())
    #a_slider.options = a_values
    #b_slider.options = b_values
    #
    #a_slider.value = a_slider.options[0]
    #b_slider.value = b_slider.options[0]

In [11]:
def change_selected_points(a, b):
    
    pc, vc, Tc, names = generate_critic_points(qgrid_table.get_selected_df())
    
    selected_113_002.x, selected_113_002.y, selected_113_002.names = vc, pc, names
    selected_113_003.x, selected_113_003.y, selected_113_003.names = vc, Tc, names

In [12]:
def update_slider_options(a_values, b_values):

    a_slider.options = a_values
    b_slider.options = b_values
    
    a_slider.value = a_values[0]
    b_slider.value = b_values[0]

In [13]:
def update_sliders(change):

    index = change.get('owner').index
    name = change.get('owner').description
    
    if name == 'a':
        b_slider.value = b_slider.options[index]
    
    elif name == 'b':
        a_slider.value = a_slider.options[index]
        
    tracer_113_002.x = np.array([scatter_113_002.x[index]])
    tracer_113_002.y = np.array([scatter_113_002.y[index]])
    
    tracer_113_003.x = np.array([scatter_113_003.x[index]])
    tracer_113_003.y = np.array([scatter_113_003.y[index]])
    
    i = a_values.index(a_slider.value)
    element_name = names_sorted[i]
    
    tracer_113_002.names = [element_name]
    tracer_113_003.names = [element_name]
    
    for mark in marks:
        if mark.labels[0] == element_name:
            mark.visible = True
        else:
            mark.visible = False
            
    
    #tracer_113_002.x, tracer_113_002.y = [fig_113_002.marks[0].x[index]], [fig_113_002.marks[0].y[index]]
    #opacities = scatter_113_003.default_opacities
    #opacities[index] = 1.0
    #fig_113_003.marks[0].default_opacities = opacities

In [14]:
def get_a_b_names(df):
    
    a_values = list(df['a (L2bar/mol2)'])
    b_values = list(df['b (L/mol)'])
    names = list(df['Element'])
    
    a_values_sorted = sorted(a_values)
    b_values_sorted = [y for _,y in sorted(zip(a_values, b_values))] #Sort values of b depending on the order of a
    names_sorted = [y for _,y in sorted(zip(a_values, names))]
    #b_values_sorted = sorted(b_values)
    
    return a_values_sorted, b_values_sorted, names_sorted

In [15]:
tt = widgets.Label("")

def hover_handler(self, content):
    tt.value = str(content.get('data').get('name'))

In [17]:
grid_options = {
    # SlickGrid options
    'fullWidthRows': True,
    'syncColumnCellResize': True,
    'forceFitColumns': True,
    'defaultColumnWidth': 150,
    'rowHeight': 28,
    'enableColumnReorder': False,
    'enableTextSelectionOnCells': True,
    'editable': False, #lehen True
    'autoEdit': False,
    'explicitInitialization': True,

    # Qgrid options
    'maxVisibleRows': 5, #we have changed it to 5 (default = 15)
    'minVisibleRows': 5, #we have changed it to 5 (default = 8)
    'sortable': True,
    'filterable': True,
    'highlightSelectedCell': False,
    'highlightSelectedRow': True
}

qgrid_table = qgrid.show_grid(database, grid_options=grid_options)

qgrid_table.on(['filter_changed'], change_visible_points)
qgrid_table.on(['selection_changed'], change_selected_points)

#[
#    'cell_edited',
#    'selection_changed',
#    'viewport_changed',
#    'row_added',
#    'row_removed',
#    'filter_dropdown_shown',
#    'filter_changed',
#    'sort_changed',
#    'text_filter_viewport_changed',
#    'json_updated'
#]

########################################
###########TOP BLOCK####################
########################################

top_block_114_000 = widgets.VBox([], layout=widgets.Layout(width='100%', align_self='center'))

a_values, b_values, names_sorted = get_a_b_names(database)

fig_113_004 = bq.Figure(title='a vs b',
                marks=[],
                axes=[],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout(height='400px')
    )

scale_x_004 = bqs.LinearScale(min = min(b_values), max = max(b_values))
scale_y_004 = bqs.LinearScale(min = min(a_values), max = max(a_values))
#a_values, b_values = get_a_b_values(database)

axis_x_004 = bqa.Axis(
                scale=scale_x_004,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='b (L/mol)',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y_004 = bqa.Axis(scale=scale_y_004,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='a (L^2 bar/mol)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_113_004.axes = [axis_x_004, axis_y_004]

scatter_113_004 = bqm.Scatter(
    name = '',
    x = b_values,
    y = a_values,
    scales = {'x': scale_x_004, 'y': scale_y_004}, 
    default_opacities = [0.2],
    visible = True,
    colors = ['#6a03a1'],
    names = names_sorted,
    display_names = False,
    labels=[],
    tooltip = tt
) 

scatter_113_004.on_hover(hover_handler)

fig_113_004.marks = [scatter_113_004]

message1 = widgets.HTML(value='<p>You can choose the points directly in the table</p>')

top_block_114_000.children = [widgets.HBox([widgets.VBox([qgrid_table, message1], layout=widgets.Layout(width='50%', margin='80px 0 0 0')),
                                    widgets.VBox([fig_113_004], layout=widgets.Layout(width='50%'))]) ]



########################################
###########MIDDLE BLOCK#################
########################################

middle_block_114_000 = widgets.HBox([], layout=widgets.Layout(width='100%', align_self='center', align_content='center'))

fig_113_002 = bq.Figure(title='pc vs vc',
                marks=[],
                axes=[],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout(width='90%')
    )
 
fig_113_003 = bq.Figure(title='Tc vs vc',
                marks=[],
                axes=[],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout(width='90%')
    )

fig_113_002.marks = []
fig_113_003.marks = []

pc, vc, Tc, names = generate_critic_points(database)


scale_x_002 = bqs.LinearScale(min = min(vc), max = max(vc))
scale_x_003 = bqs.LinearScale(min = min(vc), max = max(vc))
scale_y_p = bqs.LinearScale(min = min(pc), max = max(pc))
scale_y_T = bqs.LinearScale(min = min(Tc), max = max(Tc))


axis_x_002 = bqa.Axis(
                scale=scale_x_002,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v (L/mol)',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_x_003 = bqa.Axis(
                scale=scale_x_003,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v (L/mol)',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')



axis_y_p = bqa.Axis(
                scale=scale_y_p,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

axis_y_T = bqa.Axis(scale=scale_y_T,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='T (K)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')



fig_113_002.axes = [axis_x_002, axis_y_p]
fig_113_003.axes = [axis_x_003, axis_y_T]


tb_113_002 = Toolbar(figure=fig_113_002, layout=widgets.Layout(width='90%', display='flex'))
tb_113_003 = Toolbar(figure=fig_113_003, layout=widgets.Layout(width='90%', display='flex'))

scatter_113_002 = bqm.Scatter(
    name = '',
    x = vc,
    y = pc,
    scales = {'x': scale_x_002, 'y': scale_y_p}, 
    default_opacities = [0.2],
    visible = True,
    colors = ['#2807a3'],
    names = names,
    display_names = False,
    labels=[],
    tooltip = tt
) 

scatter_113_002.on_hover(hover_handler)

scatter_113_003 = bqm.Scatter(
    name = '',
    x = vc,
    y = Tc,
    scales = {'x': scale_x_003, 'y': scale_y_T}, 
    default_opacities = [0.2 for v in vc],
    visible = True,
    colors = ['#f5426c'],
    names = names,
    display_names = False,
    labels=[],
    tooltip = tt
) 

scatter_113_003.on_hover(hover_handler)

#fig_113_002.marks.append(scatter_113_002)
#fig_113_003.marks.append(scatter_113_003)

tracer_113_002 = bqm.Scatter(
    name = 'tracer_113_002',
    x = [1.0],
    y = [1.0],
    scales = {'x': scale_x_002, 'y': scale_y_p}, 
    default_opacities = [1.0],
    visible = True,
    colors=['black'],
    #tooltip = tt
) 

#tracer_113_002.on_hover(hover_handler)

tracer_113_003 = bqm.Scatter(
    name = 'tracer_113_003',
    x = [1.0],
    y = [1.0],
    scales = {'x': scale_x_003, 'y': scale_y_T}, 
    default_opacities = [1.0],
    visible = True,
    colors=['black'],
    #tooltip = tt,
) 

#tracer_113_003.on_hover(hover_handler)

selected_113_002 = bqm.Scatter(
    name = 'selected_113_002',
    x = [],
    y = [],
    scales = {'x': scale_x_002, 'y': scale_y_p}, 
    default_opacities = [1.0],
    visible = True,
    display_names = False,
    colors = scatter_113_002.colors,
    tooltip = tt
) 

selected_113_002.on_hover(hover_handler)

selected_113_003 = bqm.Scatter(
    name = 'selected_113_003',
    x = [],
    y = [],
    scales = {'x': scale_x_003, 'y': scale_y_T}, 
    default_opacities = [1.0],
    visible = True,
    display_names = False,
    colors = scatter_113_003.colors,
    tooltip = tt
)

selected_113_002.on_hover(hover_handler)

#fig_113_002.marks.append(tracer_113_002)
fig_113_002.marks = [scatter_113_002, selected_113_002, tracer_113_002]
fig_113_003.marks = [scatter_113_003, selected_113_003, tracer_113_003]

#a_values, b_values, names_sorted
#Use Methanol as reference: a = 9.649; b = 0.06702

a_ref = 5.536
b_ref = 0.06702
p_ref, v_ref, T_ref = calculate_critic(a_ref, b_ref)

v_values = np.linspace(1.2*b_ref, 5.0*b_ref, 300)
T_values = [0.95*T_ref, T_ref, 1.2*T_ref]
colors = ['#0079c4','#f09205','#21c400']

fig_113_005 = bq.Figure(title='p vs v',
                marks=[],
                axes=[],
                animation_duration=500, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=80, right=20),
                toolbar = True,
                layout = widgets.Layout(width='90%')
    )

scale_x_005 = bqs.LinearScale(min = min(v_values), max = max(v_values))
scale_y_005 = bqs.LinearScale(min = 0.0, max = 2.0*p_ref)

axis_x_005 = bqa.Axis(
                scale=scale_x_005,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v (L/mol)',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')


axis_y_005 = bqa.Axis(
                scale=scale_y_005,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_113_005.axes = [axis_x_005, axis_y_005]

marks = []

for i in range(len(a_values)):
    
    a = a_values[i]
    b = b_values[i]
    
    v_values = np.linspace(1.2*b, 5.0*b_ref, 300)
    
    p_values = get_absolute_isotherms(a, b, v_values, T_values)
    p_values = bar_to_atm(p_values)
    marks.append(bqm.Lines(
                    x = v_values, 
                    y = p_values, 
                    scales = {'x': scale_x_005, 'y': scale_y_005}, 
                    opacities = [1.0 for elem in p_values],
                    visible = a == a_values[0],
                    colors = colors,
                    labels = [names_sorted[i]]
    ))

fig_113_005.marks = marks    
    
middle_block_114_000.children = [widgets.VBox([fig_113_002, tb_113_002], layout=widgets.Layout(width='33%')),
                         widgets.VBox([fig_113_003, tb_113_003], layout=widgets.Layout(width='33%')),
                         widgets.VBox([fig_113_005], layout=widgets.Layout(width='33%'))
                         ]

########################################
###########BOTTOM BLOCK#################
########################################
bottom_block_114_000 = widgets.VBox([], layout=widgets.Layout(align_items='center', width='100%', margin='30px 0 0 0'))

a_slider = widgets.SelectionSlider(
    options=a_values,
    value=a_values[0],
    description='a',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    layout=widgets.Layout(width='90%'),
)

a_slider.observe(update_sliders, 'value')

b_slider = widgets.SelectionSlider(
    options=b_values,
    value=b_values[0],
    description='b',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    layout=widgets.Layout(width='90%'),
)

b_slider.observe(update_sliders, 'value')

bottom_block_114_000.children = [widgets.HBox([a_slider,
                                       widgets.HTMLMath(value=r"\( \frac{L^2 bar}{mol^2} \)",layout=widgets.Layout(height='60px') )],
                                      layout=widgets.Layout(width='50%', height='100%')),
                         widgets.HBox([b_slider,
                                       widgets.HTMLMath(value=r"\( \frac{L}{mol} \)",layout=widgets.Layout(height='60px'))],
                                      layout=widgets.Layout(width='50%'))]
########################################
###########MAIN BLOCK###################
########################################

main_block_114_000 = widgets.VBox([], layout=widgets.Layout(align_content='center'))

main_block_114_000.children = [top_block_114_000,
                               middle_block_114_000,
                               bottom_block_114_000]

main_block_114_000